In [ ]:
import pathlib as pl

import glob
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
sns.set_palette(sns.color_palette("colorblind"))

import itertools, warnings

from collections import Counter, defaultdict
from typing import List, Dict, Tuple

## Evaluating sentence-level differences between the different benchmarks


In this notebook, we will focus on trying to characterize the differences between the different benchmarks.
Before we delve into the analysis, let us define the properties we're interested in this work:
1. **words** that may (or not) be associated with gender, deemed *gender co-occurring words*; 
2. whether the **sentences are *neutral* or not**, irrespective of whether the words are associated with gender or not. We dub these sentences - *gender-invariant sentences*.


For 2., we have curated a set of prompts and conducted human evaluation to validate our claims. That is, we've characterized the distribution and showed that both pronouns fit. This validation verifies for our datasets as well as the coreference datasets, since they are created to be neutral by default.


We conjecture that by means of our prompts, the generated sentences are gender-invariant. Hence, this notebook will carry the evaluation for **1.**, i.e., as we vary the strength of association between gender and occurring words.

---


The remaining of the notebook is organized as follows: 

1. **Data loading**: we load both PMI difference values and the scores given by the models to each test sentence pair.
2. **Data analysis**: we evaluate how the benchmark size changes as we vary the strength of association between individual words and gender.
3. **Fairness metric**: we compute the fairness metric for the different models. This includes computing both the fairness metric to a fixed threshold and the AUC. 
4. **Fairness metric 2.0**: since our fairness metric focuses on the pct of "neutral" models, we aim to quantify whether the models are placing the remaining probability equally on male-female examples or whether they mostly place the probabilities on one instantiation of the templates.

## 1. Data Loading: Load PMI difference values

In [ ]:
from utils import GROUP_PAIRED_WORDLIST, FEMALE_WORDS, MALE_WORDS

In [ ]:
BASE_DIR = "/extra/ucinlp1/cbelem/projects/bias-diagnosis"

# loads the PMI information precomputed based on the PILE co-occurrence counts
GENDER_PMI = pd.read_csv(f"{BASE_DIR}/word2gender_pmi_PILE.csv", index_col=0)
print(len(GENDER_PMI))
GENDER_PMI.describe()

In [ ]:
def get_pmi_diff(df: pd.DataFrame, col1: str, col2: str, clip: int=None, missing_val: float=0, prefix_col: str="pmi__") -> pd.Series:
    """Obtains the PMI difference between columns col1 and col2. 
    
    Parameters
    ----------
    df: pandas.DataFrame
    
    col1: str
        The female word to use for computing the PMI. Should be one of the
        available suffixes in the provided dataframe's columns.
    
    col2: str
        The male word to use for computing the PMI. Should be one of the
        available suffixes in the provided dataframe's columns.
        
    clip: int, optional
        Positive integer, specifies the cap. If not specified, the pmi
        difference is only computed for words that co-occur with both
        (col1, col2). If specified, we will fill the PMI value with 0
        (ideally it would be a very negative number). You can tweak
        this value using 'missing_val'.
        
    prefix_col: str
        The prefix anteceding the col1 and col2 in the provided dataframe.
        In our files, we prefixes all columns with gendered lexicons using
        the "pmi__" prefix.
    
    Note
    ----
    To replicate the values of the paper you should pass female lexicon words
    as col1 and male lexicon words as col2.
    """
    assert f"{prefix_col}{col1}" in df.columns, f"column {col1} is undefined in dataframe"
    assert f"{prefix_col}{col2}" in df.columns, f"column {col2} is undefined in dataframe"
    
    if clip is None:
        result = df[["word", f"{prefix_col}{col1}", f"{prefix_col}{col2}"]].dropna()
    else:
        result = df[["word", f"{prefix_col}{col1}", f"{prefix_col}{col2}"]].fillna(missing_val)
        
    print(f"('{col1}', '{col2}'): {len(result)}")
    result[f"pmi({col1})-pmi({col2})"] = result[f"{prefix_col}{col1}"] - result[f"{prefix_col}{col2}"]
    
    if clip is not None:
        result[f"pmi({col1})-pmi({col2})"].clip(lower=-clip, upper=clip, inplace=True)
    return result


def get_gender_pairs_matrix(gender_pmi_df: pd.DataFrame, parallel_terms: list, **kwargs):
    # dataframe with all the group pairs PMI (per word)
    # (words for which no PMI diff is define)
    pairs = gender_pmi_df[["word"]].copy().set_index("word")
    num_words = []

    for fword, mword in parallel_terms:
        try:
            # Compute the pmi difference between fword and mword
            d = get_pmi_diff(gender_pmi_df, fword, mword, **kwargs).set_index("word")
            # Rename to be easier to visualize
            d = d.rename({f"pmi({fword})-pmi({mword})": f"{fword}-{mword}"}, axis=1)
            # Number of well-defined words for each of the gender pairs
            num_words.append((f"{fword}-{mword}", len(d)))
            pairs = pairs.join(d[[f"{fword}-{mword}"]])
        except:
            print(f"! Pair ({fword}, {mword}) doesn't exist...")

    return pairs, num_words


# Since we may want to perform some correlation with other gendered words
# we also define the PMI diff between words and other gendered word pairs
GENDER_PAIRS, GENDER_PAIRS_NUM_WORDS = get_gender_pairs_matrix(GENDER_PMI, GROUP_PAIRED_WORDLIST)

# ----------------------------------------------------------------------------
# compute PMI diff used in the main paper
# ----------------------------------------------------------------------------
# Most analysis will focus on the pmi_diff(she, he)
PMI_DIFF = get_pmi_diff(GENDER_PMI, "she", "he").sort_values("pmi(she)-pmi(he)")
# rename pmi difference column to be something less verbose :b
PMI_DIFF = PMI_DIFF.rename({"pmi(she)-pmi(he)": "pmi_diff"}, axis=1)
PMI_DIFF.sample(15, random_state=81273)

## 2. Loading data - Load model scores for the different datasets

Say, PMI_DIFF(w, she, he), let us now compute the pmi of the words used for each of the benchmarks.

In [ ]:
BASE_DIR = ".."

# list all the score files per dataset
DATASET_2_FILEPATHS = {
    "Ours-5": glob.glob(f"{BASE_DIR}/results-words5/intervention-results/*__scores__*.csv"),
    # Baselines below ----
    "Winobias": glob.glob(f"{BASE_DIR}/results-baselines/intervention-results/*Winobias*__scores__*.csv"),
    "Winogender": glob.glob(f"{BASE_DIR}/results-baselines/intervention-results/*Winogender*__scores__*.csv"),
    # "StereoSet": glob.glob(f"../results-baselines/final-results/*StereoSet*__scores__*.csv"),
    # We define this ordering so that we can automatically obtain the same coloring scheme as
    # the one used for word analysis
    "Ours-10": glob.glob(f"{BASE_DIR}/results-words10/intervention-results/*__scores__*.csv"),
    "Ours-20": glob.glob(f"{BASE_DIR}/results-words20/intervention-results/*__scores__*.csv"),
}

DATASET_NAMES = list(DATASET_2_FILEPATHS.keys())
print(" Dataset names:\n  ->", DATASET_NAMES, "\n", "-" * 62)

# ------------------------------------------------------------------------------
#                                     Validation
# ------------------------------------------------------------------------------
# All datasets must have exact same number of files and ordered in the same way.
for dataset1, dataset2 in itertools.product(DATASET_NAMES, DATASET_NAMES):
    fps1 = [fp.rpartition("__scores__")[-1] for fp in DATASET_2_FILEPATHS[dataset1]] 
    fps2 = [fp.rpartition("__scores__")[-1] for fp in DATASET_2_FILEPATHS[dataset2]] 
    c1, c2 = Counter(fps1), Counter(fps2)
    assert len(c1 & c2) == len(c1), f"Validation failed for datasets: ({dataset1}, {dataset2})"

# !! Assumption: When scoring there was no change in the ordering of the templates and therefore
# every time we load the filepaths, we will have exactly the same ordering for all files (regardless
# of the scoring model).
DATASET_2_FILEPATHS = {k: sorted(v) for k, v in DATASET_2_FILEPATHS.items()}

print(" Number of files per dataset", "\n", "-" * 65)
for name, files in DATASET_2_FILEPATHS.items():
    print(" -> ", name, len(files))

#### Preprocess the datasets

Transform the datasets into the canonic form:

1. Transform model name into its canonic form: Extract from filepath name and add it as a column to the dataset.
3. Obtain information about model size: 
2. Obtain information about the interventions: Is the model trained on duplicated data (is_deduped=False) or non-duplicated data (is_deduped=True).
3. Obtain information about whether the test sentence pair is natural (is_natural=True) or whether is unnatural for one of the variants in the pair (is_natural=False)
4. Obtain information about the model family.

In [ ]:
def canonic_model_name(model_name: str) -> str:
    if "EleutherAI__" in model_name:
        model_name = model_name.replace("EleutherAI__", "")
    elif "facebook__" in model_name:
        model_name = model_name.replace("facebook__", "")
    elif "llama" in model_name:
        ix = model_name.index("llama")
        model_name = model_name[ix:].replace("__hf_models__", "-")
    elif "mosaicml__" in model_name:
        model_name = model_name.replace("mosaicml__", "")
        
    if "deduped" in model_name:
        model_name = model_name.replace("-deduped", " (D)")
    return model_name


def get_model_size(canonic_name: str) -> int:
    import re 
    val = re.search(r"(\d+(\.\d+)?)(b|B|m|M)", canonic_name)[0]
    const = 1_000 if val[-1] in ("b", "B") else 1        
    return float(val[:-1]) * const
        
    
def get_model_family(model_name: str) -> str:
    """Collects information about the model family"""
    if "pythia" in model_name:
        return "pythia"
    elif "opt" in model_name:
        return "opt"
    elif "mpt" in model_name:
        return "mpt"
    elif "llama" in model_name:
        return "llama2"
    elif "gpt" in model_name:
        return "gpt-j"

    
def is_deduped(model_name: str) -> bool:
    """Collect information about whether the model was trained on deduplicated data."""
    return True if '-deduped' in model_name else False
    

def is_intervention(model_name: str) -> bool:
    """Collect information about whether the model was trained on deduplicated data 
    and with gender bias intervention.
    """
    return True if '-intervention' in model_name else False


def get_model_step(name: str) -> int:
    model_name = name.rpartition("__step")[-1]
    return int(model_name)


def get_model_intervention(name: str) -> str:
    if "intervention" in name:
        return "intervention"
    elif "deduped" in name:
        return "deduped"
    else:
        return "other"


def remove_unnatural_examples(df: pd.DataFrame) -> pd.DataFrame:
    """Filter out unnatural examples from the provided dataframe.
    
    Natural test sentence pairs are those for which ChatGPT
    indicates that both sentence variants (regardless of gender)
    are both likely to occur. If one of them is unlikely (as per
    ChatGPT prediction) then we will deem the whole test sentence
    pair unnatural and remove it.
    
    The proposed datasets were generated from scratch and therefore
    will be the only ones with this column. The WinoBias and Winogender
    have no such information, since we know by definition that both
    completions of the sentences are both likely.
    """
    if "is_natural" in df.columns:
        return df[df["is_natural"]].reset_index(drop=True)
    return df

def read_filepath(fp: str, dataset: str, filter_unnatural: bool) -> pd.DataFrame:
    df = pd.read_csv(fp)
    # df has "model" information, with the fully qualified name (including company name)

    # add dataset name to dataframe
    df["dataset"] = dataset
    # add boolean identifying whether model was trained on deduplicated data
    df["is_deduped"] = df["model"].apply(is_deduped)
    # add boolean identifying whether it is gender swap intervention
    df["is_gender_swap"] = df["model"].apply(is_intervention)
    # categorical stating what type of intervention it is
    df["intervention_type"] = df["model"].apply(get_model_intervention)
    # model step
    df["step"] = df["model"].apply(get_model_step)

    # add boolean indentifying whether model was trained with gender swap
    df["is_intervention"] = df["model"].apply(is_intervention)
    # add canonic name (no company name, with size info)
    df["orig_model_name"] = df["model"]
    df["model"] = df["model"].apply(canonic_model_name)
    # add model size (as a float)
    df["model_size"] = df["model"].apply(get_model_size)
    # add model family
    df["model_family"] = df["model"].apply(get_model_family)

    # add information about whether templates are likely or unlikely
    if filter_unnatural:
        #print("Before filtering unnatural:", len(df))
        df = remove_unnatural_examples(df)
        #print("After filtering unnatural:", len(df))
    
    df = df.reset_index(names=["orig_index"])
    return df


# Mapping from dataset name to the file dataframes
DATASET_2_FILES = defaultdict(list)

# Read each individual filepath, creating an association <str, list<dataframe>>.
# every str should have a list of the same size.
DATASET_2_FILES = {
    dataset: [read_filepath(fp, dataset, filter_unnatural=True) for fp in sorted(fps)]
    for dataset, fps in DATASET_2_FILEPATHS.items()
}

# Merge all the dataframes into a single big dataframe that contains the information of all models
# for each dataset. We've created a original index to keep track of the unique sentences.
# Sort the files per (model, orig_index)
DATASET_2_FILES = {
    dataset: pd.concat(dfs).sort_values(["model", "orig_index"]).reset_index(drop=True)
    for dataset, dfs in DATASET_2_FILES.items()
}

# Number of models being evaluated 
NUM_EVAL_MODELS = []
MODELS = []
for dataset, df  in DATASET_2_FILES.items():
    print(dataset, df["model"].nunique())
    MODELS.extend(df["model"].unique())
    NUM_EVAL_MODELS.append(df["model"].nunique())
    
# We force the number of models to be the same across all datasets
assert len(set(NUM_EVAL_MODELS)) == 1, \
    f"Found various model sizes: {NUM_EVAL_MODELS}"

NUM_EVAL_MODELS = NUM_EVAL_MODELS[0]
print("Evaluating", NUM_EVAL_MODELS, "models:")
MODELS = list(sorted(set(MODELS)))
print(" -", "\n - ".join(MODELS))

DATASET_2_FILES["Ours-5"]

## Post processing: 

In this section, we will carry some processing of the templates (column "template").


**1. Remove placeholders from templates** : We first remove the placeholders (e.g., "{PRONOUN}", "{PRONOUN1}", "{PRONOUN2}", "{PRONOUN2}self") from the template.

**2. Remove stopwords from the templates**: We use **spacy**'s stopwords except that we add back some of the pronouns, effectively following the approach in [Razeghi et al 2022](https://aclanthology.org/2022.emnlp-demos.39/).

**3. Parse each template**: We use **spacy** tokenizer since this was what was used by [Razeghi et al 2022](https://aclanthology.org/2022.emnlp-demos.39/). While NTLK is much faster, it doesn't group together words like "self-care", which is treated as single word by spacy tokenizer. Therefore, we've updated the script to consider the spacy tokenization. Applying it to the whole DATASET_2_FILES[dataset] will be too time-consuming, so we will apply to the first portion of the data and then concatenate it to the dataframe.


### Filtering


Before applying the processing, we will first obtain the top unique templates by focusing on the subset of data of the first listed model.

In [ ]:
# Validation (!sanity check)
# When selecting a data slice from the big dataframe
# we must guarantee that the sentences match each other

def check_slices(dataset, data2files: dict, models: List[str]):
    slices = []
    for model in models:
        df = data2files[dataset]
        df = df[df["model"] == model].copy()
        
        if len(slices) > 1:
            assert np.array_equal(slices[-1]["template"].values, df["template"].values) 
            
        slices.append(df)
        
    
for dataset in DATASET_NAMES:
    print("Checking slices for dataset:", dataset)
    check_slices(dataset=dataset, data2files=DATASET_2_FILES, models=MODELS)
    
# -----------------------------------------------------------------------------
# ^Note: if the check above does not throw an error, then it means that the
# templates can stack up based on the model, so it's ok to apply the processing
# to the first model and then create NUM_EVAL_MODEL copies of that and insert
# in the dataframe!!
# -----------------------------------------------------------------------------
DATASET_2_TEMPLATES = {
    dataset: df[df["model"] == MODELS[0]]["template"].values.tolist()
    for dataset, df in DATASET_2_FILES.items()
}

### Processing (using Spacy tokenizer)

In [ ]:
import re, spacy

#!python -m spacy download en_core_web_md
SPACY_PARSER = spacy.load("en_core_web_md", disable=["ner", "tagger"])

def word_tokenize(sentence: str, pronouns: list, remove_stopwords: bool=True, remove_punct: bool=True):
    doc = SPACY_PARSER(sentence)
    # Extract the tokens that are not stopwords
    tokens = [token.text for token in doc 
              if (token.text in pronouns) or (not token.is_stop and not token.is_punct)]
    return [t for t in tokens if len(t.strip()) > 0]

In [ ]:
PRONOUNS = ["she", "her", "hers", "he", "his", "him", "himself", "herself"]


def postprocess_spacy(templates, pronouns=PRONOUNS):
    templates = [t.lower() for t in templates]
    # Step 1. Remove placeholders from teh templates
    templates = [t.replace("{pronoun2}self", "") for t in templates]
    templates = [re.sub(r"\{pronoun([0-2]{1})?\}", "", t) for t in templates]
    # Step 2. Parse the sentence
    templates = [word_tokenize(t, pronouns) for t in templates]
    return templates


def postprocess_nltk(templates, pronouns=PRONOUNS):
    import nltk 
    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize
    import string, re
    nltk.download('stopwords')
    nltk_stopwords = set(stopwords.words('english'))
    # We know that some sentences have some other references to other entities,
    # let's keep some pronouns
    nltk_stopwords -= set(pronouns)
    punct = string.punctuation
    
    templates = [t.lower() for t in templates]
    # Remove pronouns first
    templates = [t.replace("{pronoun2}self", "") for t in templates]
    templates = [re.sub(r"\{pronoun([0-2]{1})?\}", "", t) for t in templates]
    
    # Remove stopwords and punct
    templates = [[w for w in word_tokenize(t) if w not in punct and w not in nltk_stopwords] for t in templates]
    return templates


DATASET_2_CANONIC_TEMPLATES_SPACY = {}
DATASET_2_CANONIC_TEMPLATES_NLTK = {}

for dataset, templates in DATASET_2_TEMPLATES.items():
    DATASET_2_CANONIC_TEMPLATES_SPACY[dataset] = postprocess_spacy(templates)
    DATASET_2_CANONIC_TEMPLATES_NLTK[dataset] = postprocess_nltk(templates)

## Data Analysis - Determine gender co-occurrence for each word 

In this section, we iterate the templates and compute the gender co-occurrence values for each sentence in the benchmarks. Optionally, you can weight the values of each word by the likelihood of being toxic or having negative sentiment. If such values are not provided, we assume each word is worth the same value of 1 unit.


In [ ]:
## Convert dataframe to mapping from word to pmi diff for easy access
WORD2PMI = PMI_DIFF[["word", "pmi_diff"]].set_index("word").to_dict()["pmi_diff"]
WORD2WEIGHTS = defaultdict(lambda: 1)

## ----------------------------------------------------------------
## Weighting words based on frequency
## ----------------------------------------------------------------
FREQ_WORDS = pd.read_csv("./pmi_file_w_counts.csv", index_col=0)
FREQ_WORDS["log_freq"] = np.log(FREQ_WORDS["freq"])

## uncomment one of the lines below if you prefer weighting each word based
## on the frequency of each individual word
# WORD2WEIGHTS = FREQ_WORDS[["word", "freq"]].set_index("word").to_dict()["freq"]
# WORD2WEIGHTS = FREQ_WORDS[["word", "log_freq"]].set_index("word").to_dict()["log_freq"]

## ----------------------------------------------------------------
## Weighting words based on toxicity/sentiment
## ----------------------------------------------------------------
## TODO:
## -> Define toxicity for each word
## -> Define sentiment polarity for each word (?)
## Define a 1-to-1 mapping and assign the variable WORD2WEIGHTS

In [ ]:
def compute_pmi_diff_per_sentences(
        templates: List[List[str]],
        word2pmi: dict,
        word2weights: dict,
    ) -> List[List[float]]:
    """Computes the PMI difference per individual token in the provided sentences.
    
    Notes
    -----
    It assumes the templates/sentences are already provided as a list of tokens.
    It returns two lists: the first one contains the list of pmi values for each of
    the provided words (some tokens won't have a PMI value associated); the second
    list contains the 1-1 mapping from word to pmi value and their weights.
    """
    pmi_values = []
    words_with_pmi = []
    
    for template in templates:
        pmi = np.array([word2weights[w] * word2pmi.get(w) for w in template if word2pmi.get(w) is not None])
        pmiwords = [{
            "word": w, 
            "pmi": round(word2pmi.get(w), 2),
            "weight": round(word2weights[w], 2),
        } for w in template if word2pmi.get(w) is not None]
        
        pmi_values.append(pmi)
        words_with_pmi.append(pmiwords)
            
    return pmi_values, words_with_pmi
        

    
    
PMI_PER_SENTENCES_NLTK = {dataset: 
                          compute_pmi_diff_per_sentences(templates, WORD2PMI, WORD2WEIGHTS)
                          for dataset, templates in DATASET_2_CANONIC_TEMPLATES_NLTK.items()
}

PMI_PER_SENTENCES_SPACY = {dataset: 
                          compute_pmi_diff_per_sentences(templates, WORD2PMI, WORD2WEIGHTS)
                          for dataset, templates in DATASET_2_CANONIC_TEMPLATES_SPACY.items()
}

# plot distribution
for dataset in DATASET_NAMES:
    # Compute the number of well-defined tokens with each of the approaches
    nltk_len = list(map(len, PMI_PER_SENTENCES_NLTK[dataset][0]))
    spacy_len = list(map(len, PMI_PER_SENTENCES_SPACY[dataset][0]))
    
    plt.figure(figsize=(8, 6))
    plt.title(dataset)
    sns.histplot(list(nltk_len), label="nltk", stat="probability", alpha=0.6, binrange=(0,20), binwidth=1)
    sns.histplot(list(spacy_len), label="spacy", stat="probability", alpha=0.6, binrange=(0,20), binwidth=1)
    plt.legend()
    plt.show()

Since in general **spacy** tokenizer leads to higher pct of examples being matched with a word. We will use the **spacy** tokenized templates to conduct the analysis (it increases the coverage of the constraints).

In [ ]:
PMI_PER_TEMPLATES = {}
PMIWORDS_PER_TEMPLATES = {}

# Change the PMI_PER_SENTENCES_SPACY with PMI_PER_SENTENCES_NLTK
# to use NLTK tokenization instead.
for dataset, pmi_per_sents_values in PMI_PER_SENTENCES_NLTK.items():
# for dataset, pmi_per_sents_values in PMI_PER_SENTENCES_SPACY.items():
    pmi_vals, words_per_pmi = pmi_per_sents_values
    
    PMI_PER_TEMPLATES[dataset] = pmi_vals
    PMIWORDS_PER_TEMPLATES[dataset] = words_per_pmi

### Compute the constraint: MaxGenderPMI(s)

In this section, we compute the max gender PMI value per sentence. 

In [ ]:
MAXGENDER_COL = "max_gender_pmi"

def max_gender_pmi(templates_pmi: List[List[str]], col: str) -> List[dict]:
    """Compute the maximum PMI diff per sentence."""
    def _max_pmi(lst_pmis: List[str]) -> float:
        if len(lst_pmis) > 0:
            idx = np.argmax(np.abs(lst_pmis))
            return lst_pmis[idx]
    
    results = []
    for template_pmi in templates_pmi:
        max_val = _max_pmi(template_pmi)
        results.append({col: max_val, f"{col}_invalid": max_val is None, "template_words_pmi": template_pmi})
        
    return results

In [ ]:
# Contains the max gender pmi values per sentence
MAX_GENDER_PMI = {dataset: max_gender_pmi(templates_pmi, MAXGENDER_COL) 
                  for dataset, templates_pmi in PMI_PER_TEMPLATES.items()}

MAX_GENDER_PMI_LONG = []
for dataset, lst_value_dicts in MAX_GENDER_PMI.items():
    for value_dict in lst_value_dicts:
        r = {k: v for k, v in value_dict.items()}
        r["dataset"] = dataset
        MAX_GENDER_PMI_LONG.append(r)

MAX_GENDER_PMI_LONG = pd.DataFrame(MAX_GENDER_PMI_LONG)
        
# Adds the information to the original dataset with all models
# originally, preserved in the variable DATASET_2_FILES
DATASET_W_CONSTRAINTS = {dataset: pd.DataFrame(values * NUM_EVAL_MODELS)
                  for dataset, values in MAX_GENDER_PMI.items()}

# ------------------------------------------------------------------------------
# 
#                        Dataset w/ MaxGender PMI constraint!
# 
# ------------------------------------------------------------------------------
DATASET_W_CONSTRAINTS = {
    dataset: pd.concat((DATASET_2_FILES[dataset], DATASET_W_CONSTRAINTS[dataset]), copy=True, axis=1)
    for dataset in DATASET_NAMES
}

DATASET_W_CONSTRAINTS[DATASET_NAMES[0]].columns

In [ ]:
invalid_results = {dataset: df[df[f"{MAXGENDER_COL}_invalid"]] for dataset, df in DATASET_W_CONSTRAINTS.items()}
invalid_results

In [ ]:
# Keep only the sentences w/ valid constraints
DATASET_W_CONSTRAINTS = {d: df[~df[f"{MAXGENDER_COL}_invalid"]] for d, df in DATASET_W_CONSTRAINTS.items()}


In [ ]:
MAX_GENDER_PMI_LONG[["dataset", MAXGENDER_COL]].groupby("dataset").describe().T[DATASET_NAMES].style.format('{:.2f}')

In [ ]:
MAX_GENDER_PMI_LONG["dataset_"] = MAX_GENDER_PMI_LONG.dataset.apply(lambda x: "WB" if x == "Winobias" else ("WG" if x == "Winogender" else x))

plt.figure(figsize=(4, 3), dpi=200)
sns.boxplot(MAX_GENDER_PMI_LONG, x="dataset_", y=MAXGENDER_COL)
plt.ylim(-3, 3)
plt.ylabel("Gender MaxPMIDiff")
plt.xlabel(None)
plt.show()

In [ ]:
num_templates = MAX_GENDER_PMI_LONG.groupby("dataset").count()[[MAXGENDER_COL]]
num_templates.rename({MAXGENDER_COL: "orig_num_templates"}, axis=1, inplace=True)
num_templates

## Data Analysis - Filtering using $\epsilon_k$

In this section, we observe how the number of templates changes as we increase the max gender pmi difference. We observe that little to no evaluation examples remain after enforcing smaller values of MaxGenderPMIDiff. Conversely, as we relax the constraint, more and more examples are included.

In [ ]:
def filter_epsilon_k(
        data: dict,
        epsilons: List[float],
        col: str,
        constant: int=NUM_EVAL_MODELS,
    ) -> pd.DataFrame:
    results = defaultdict(list)
    
    dataset_max_counts = defaultdict(lambda: 0)
    for eps in epsilons:
        for dataset, df in data.items():
            counts = ((df[col] >= -eps) & (df[col] <= eps)).sum() / constant
            results["dataset"].append(dataset)
            results["filter"].append(eps)
            results["counts"].append(counts)
            
            if dataset_max_counts[dataset] < counts:
                dataset_max_counts[dataset] = counts
            
    results = pd.DataFrame(results)
    results["freq"] = results[["dataset", "counts"]].apply(lambda x: x["counts"]/(dataset_max_counts[x["dataset"]]), axis=1)
    
    return pd.DataFrame(results)


FILTERING_EPSILONS = np.linspace(0.10, 2.5, 101)[::-1]

FILTER_CURVES_RESULTS = filter_epsilon_k(
    DATASET_W_CONSTRAINTS,
    FILTERING_EPSILONS,
    MAXGENDER_COL,
    NUM_EVAL_MODELS)

plt.figure(figsize=(4,3), dpi=200)
sns.lineplot(FILTER_CURVES_RESULTS, x="filter", y="freq", hue="dataset", lw=1) #set y="counts" to plot absolute values instead
plt.xlabel("Gender co-occurrence threshold ($\epsilon_k$)")
plt.ylabel("% of the dataset")
#plt.legend( loc='upper left', bbox_to_anchor=(1, 1.01))
plt.tight_layout()
plt.show()

## Fairness metrics - Fixed threshold & AUC


In [ ]:
FAIRNESS_COL = "FM_logprob"
FAIRNESS_THRESHOLD = 0.5
FAIRNESS_EPSILONS = np.linspace(0, 10, 101)
from sklearn.metrics import auc

def compute_neutralpct_fixed_threshold(dataset: pd.DataFrame, eps: float, col: str):
    abs_col = dataset[col].apply(np.abs)
    counts = (abs_col <= eps).sum()
    freq = counts / len(dataset)
    
    return counts, freq


def compute_neutralpct_auc(dataset: pd.DataFrame, epsilons: List[float], col: str):
    results = defaultdict(list)
    
    for eps in epsilons:
        counts, freq = compute_neutralpct_fixed_threshold(dataset, eps, col)
        results["fairness_eps"].append(eps)
        results["num_examples"].append(counts)
        results["pct_examples"].append(freq)
        
    results = pd.DataFrame(results)    
    return results, auc(results["fairness_eps"], results["pct_examples"])


def compute_neutralpct(data: dict, models: List[str], datasets: List[str], epsilons: List[float], col: str):
    results = []
    results_auc = defaultdict(list)

    for dataset in datasets:
        df = data[dataset].copy()
        
        for model in models:
            df_model = df[df["model"] == model].copy()
            out, out_auc = compute_neutralpct_auc(df_model, epsilons, col)
            
            out["model"] = model
            out["is_gender_swap"] = is_intervention(model)
            out["step"] = get_model_step(model)
            out["dataset"] = dataset
            results.append(out)
            
            results_auc["dataset"].append(dataset)
            results_auc["model"].append(model)
            results_auc["is_gender_swap"].append(is_intervention(model))
            results_auc["step"].append(get_model_step(model))
            results_auc["auc"].append(out_auc)
            
            
    return pd.concat(results), pd.DataFrame(results_auc)

In [ ]:
FAIR_THRESHOLDS, FAIR_AUC = compute_neutralpct(DATASET_W_CONSTRAINTS, MODELS, DATASET_NAMES, FAIRNESS_EPSILONS, FAIRNESS_COL)

In [ ]:
sns.boxplot(FAIR_AUC, x="dataset", y="auc")
plt.axhline(10, ls="--", color="black", label="max auc")
plt.xlabel(None)
plt.ylabel("Area under the fairness curve ($\epsilon_f \in [0, 10]$)")

In [ ]:
sns.lineplot(FAIR_AUC, x="step", y="auc", hue="dataset", style="is_gender_swap", lw=1)

### Fairness AUC table

In [ ]:
FAIR_AUC["dataset_"] = FAIR_AUC["dataset"].apply(lambda x: x if x != "Ours-5" else "Ours-05")
pd.pivot_table(FAIR_AUC, index="model", values=["auc"], columns=["dataset_"]).style.format('{:.2f}')

In [ ]:
def fairness_threshold_plots(fairthresholds, fairauc, datasetnames, models):
    for dataset in datasetnames:
        ft_df = fairthresholds[fairthresholds["dataset"] == dataset]
        ft_df = ft_df[ft_df["model"].isin(models)]


        aucs = fairauc[(fairauc["dataset"] == dataset) & (fairauc["model"].isin(models))]

        ft_df["is_deduplicated"] = ft_df["model"].apply(lambda x: "(D)" in x)
        ft_df["model"] = ft_df["model"].apply(lambda x: x.replace(" (D)", ""))
        print(aucs)
        plt.figure(figsize=(4,3), dpi=200)
        sns.lineplot(ft_df, x="fairness_eps", y="pct_examples", hue="model", style="is_deduplicated", lw=1)
        plt.axvline(FAIRNESS_THRESHOLD, color="gray", alpha=0.5)
        plt.tight_layout()
        plt.title(dataset)
        plt.xlabel("fairness threshold ($\epsilon_f$)")
        plt.ylabel("fairness metric ($\\tau$)")

        plt.show()

### AuFC: Pythia models

In [ ]:
MODELS

In [ ]:
pythia_models = [
    'pythia-6.9b (D)__step143000',
    'pythia-intervention-6.9b (D)__step143000',
]

fairness_threshold_plots(FAIR_THRESHOLDS, FAIR_AUC, DATASET_NAMES, pythia_models)

## Fairness Neutrality

In this section, we aim to compute the different skews of the models for various constrained settings. 

In particular, we will compute:

1. Fairness metric: focus on the computation of the neutral examples, i.e., the examples whose test sentence pair likelihoods are within $\exp^{\epsilon_f}$
2. Difference in predicted female vs predicted male: if the sentences are not being predicted neutral, how is the model assigning the probability? 

In [ ]:
FAIRNESS_THRESHOLD = 0.5
PMI_THRESHOLDS = [0.5, 0.65, 0.8, 1.0]

def filter_data_by_col_val(data: pd.DataFrame, col: str, thres: float):
    return data[(data[col] >= -thres) & (data[col] <= thres)]


# Compute the constrained version of the file
BEFORE_FILTER = {dataset: df.copy() for dataset, df in DATASET_W_CONSTRAINTS.items()}
AFTER_FILTER = {}

# Filter out the dataset_w_constraints according to the different PMI thresholds (or \epsilon_k)
for pmi_threshold in PMI_THRESHOLDS:
    AFTER_FILTER[pmi_threshold] = {
        dataset: filter_data_by_col_val(df, col=MAXGENDER_COL, thres=pmi_threshold).copy()
        for dataset, df in BEFORE_FILTER.items()
    } 

In [ ]:
import glob, operator


def is_neutral(df, col=FAIRNESS_COL, threshold: float=FAIRNESS_THRESHOLD):
    assert 0 <= threshold <= 1
    assert col in df.columns
    return (df[col] >= -threshold) & (df[col] <= threshold)


def get_skew(df, col=FAIRNESS_COL, threshold: float=FAIRNESS_THRESHOLD):
    assert 0 <= threshold <= 1
    assert col in df.columns

    df = df.copy()
    df["skew"] = ["neutral"] * len(df)
    df.loc[df[col] < -threshold, "skew"] = "male"
    df.loc[df[col] >  threshold, "skew"] = "female"
    return df["skew"]


def get_bins(val, max_val=100, edges=(15, 10, 5, 2.5, 1, FAIRNESS_THRESHOLD)):
    __base_interval = pd.Interval(-edges[-1], edges[-1], closed="both")
    sign = np.sign(val)
    threshold = edges[-1]

    if sign == 0 or  -threshold <= val <= threshold:
        return __base_interval

    op = operator.gt if sign > 0 else operator.le
    edges = [sign * max_val] + [e * sign for e in edges]


    for i in range(1, len(edges)):
        if op(val, edges[i]):
            e1, e2 = edges[i-1], edges[i]
            bins = (e1, e2) if sign < 0 else (e2, e1)
            return pd.Interval(*bins, closed="neither" if sign < 0 and bins[-1] == -threshold else "right")
        
        
        
def compute_skews_(data_files: dict, fairness_col, fairness_threshold):
    # Note: This cell is going to add columns to the original dataframes in DATASET_2_FILES
    for name, df in data_files.items():
        get_fair_bins = lambda x: get_bins(val=x, max_val=100, edges=(15, 10, 5, 2.5, 1, fairness_threshold))
        df[f"{fairness_col}_bins"] = df[fairness_col].apply(get_fair_bins)

        df["is_neutral"] = is_neutral(df, fairness_col, fairness_threshold)
        # Obtain a discrete measure of what gender does the model fairness_col, skews
        # note: it assumes that positive values of fairness col will skew female
        # completions; and negative values skew male completions...
        df["skew"] = get_skew(df, fairness_col, fairness_threshold)
        
    return data_files


BEFORE_FILTER = compute_skews_(BEFORE_FILTER, FAIRNESS_COL, FAIRNESS_THRESHOLD)

AFTER_FILTER = {
    filt: compute_skews_(bias_files, FAIRNESS_COL, FAIRNESS_THRESHOLD) for filt, bias_files in AFTER_FILTER.items()
}

### Neutrality and AuFC (per constrained setting)

In [ ]:
def compute_neutral_pct_w_std(data2files: dict):
    results = defaultdict(list)
    for dataset, df in data2files.items():
        neutral_mean = df[["model", "is_neutral"]].groupby("model").mean()
        neutral_mean *= 100

        # computed as the variance of a bernoulli distribution
        Y = neutral_mean

        n = len(BEFORE_FILTER["Ours-5"]) / NUM_EVAL_MODELS
        neutral_std = np.sqrt(Y/100 * (1 - Y/100) / n) * 100
        # neutral_std = BEFORE_FILTER["Ours-5"][["model", "is_neutral"]].groupby("model").std()

        results["dataset"].extend([dataset if dataset != "Ours-5" else "Ours-05"] * len(neutral_mean))
        results["model"].extend(neutral_mean.reset_index()["model"])
        results["neutral_avg"].extend(neutral_mean["is_neutral"].values.tolist())
        results["neutral_std"].extend(neutral_std["is_neutral"].tolist())
        final_repr = "$" + neutral_mean["is_neutral"].map('{:.2f}'.format) + "_{\\pm " + neutral_std["is_neutral"].round(2).map('{:.2f}'.format) + "}$"

        results["neutral_final"].extend(final_repr.values.tolist())
        
    return pd.DataFrame(results)
        
    
def compute_female_male_skews(data2files: dict, model_names=MODELS):
    results = defaultdict(list)
    for dataset, df in data2files.items():
        pcts = df.groupby(["model", "skew"]).count()["template"]
        
        for model in model_names:
            model_res = pcts[model]
            model_total = model_res.sum()
            
            results["dataset"].append(dataset if dataset != "Ours-5" else "Ours-05")
            results["model"].append(model)
            results["total"].append(model_total)
            results["pct_fem"].append(model_res.get("female", 0) / model_total * 100)
            results["pct_mal"].append(model_res.get("male", 0) / model_total * 100)
            
            pct_diff = round((model_res.get("female",0) - model_res.get("male", 0)) / model_total * 100, 2)
            results["pct_fem_min_mal"].append(f"{pct_diff:.2f}")
           
    return pd.DataFrame(results).round(2)


def merge_results(data2files) -> pd.DataFrame:
    return pd.merge(
        compute_neutral_pct_w_std(data2files),
        compute_female_male_skews(data2files),
        on=["dataset", "model"],
        how="inner"
    )


In [ ]:
METRICS_BEFORE_FILTER = merge_results(BEFORE_FILTER)
METRICS_AFTER_FILTER = {eps: merge_results(AFTER_FILTER[eps]) for eps in AFTER_FILTER.keys()}

In [ ]:
METRICS_AFTER_FILTER[0.5]

In [ ]:
{dataset: len(df) / NUM_EVAL_MODELS for dataset, df in BEFORE_FILTER.items()}
{dataset: len(df) / NUM_EVAL_MODELS for dataset, df in AFTER_FILTER[0.5].items()}

#{dataset: len(df) / NUM_EVAL_MODELS for dataset, df in AFTER_FILTER[0.65].items()}
#{dataset: len(df) / NUM_EVAL_MODELS for dataset, df in AFTER_FILTER[0.80].items()}

In [ ]:
def model2latex(model: str):
    if "pythia" in model:
        return "\\" + re.sub(r"pythia-(.+)", r"pyths{\1}", model)
    elif "opt" in model:
        return "\\" + re.sub(r"opt-(.+)", r"opts{\1}", model)
    elif "mpt" in model:
        return "\\" + re.sub(r"mpt-(.+)", r"mpts{\1}", model)
    elif "llama-2" in model:
        return "\\" + re.sub(r"llama-2-(.+)", r"llamas{\1}", model)
    elif "gpt-j" in model:
        return "\\" + "gptj"
    else:
        raise ValueError(f"Unexpected: {model}")
        
        
def print_results(data, value):
    table = pd.pivot(data, values=[value], index="model", columns=["dataset"])
    table = table.droplevel(None, axis=1).rename_axis(None, axis=1).reset_index() 
    table["model"] = table["model"].apply(model2latex)
    print(table.set_index("model").to_latex())

    
def get_results(data, value):
    table = pd.pivot(data, values=[value], index="model", columns=["dataset"])
    table = table.droplevel(None, axis=1).rename_axis(None, axis=1).reset_index() 
    table["model"] = table["model"].apply(model2latex)
    return table.set_index("model")

### Neutral fairness

In [ ]:
print("-" * 80, "\n")
print("NO FILTER")
print("\n", "-" * 80, "\n\n")
print_results(METRICS_BEFORE_FILTER, "neutral_final")



for eps, df in METRICS_AFTER_FILTER.items():
    print("-" * 80, "\n")
    print(f"FILTER = {eps}")
    print_results(METRICS_AFTER_FILTER[eps], "neutral_final")
    print("-" * 80, "\n\n")

### pred female - pred male

In [ ]:
get_results(METRICS_BEFORE_FILTER, "pct_fem_min_mal")
#get_results(METRICS_AFTER_FILTER[0.8], "pct_fem_min_mal")
#get_results(METRICS_AFTER_FILTER[0.65], "pct_fem_min_mal")
#get_results(METRICS_AFTER_FILTER[0.5], "pct_fem_min_mal")

In [ ]:
print("-" * 80, "\n")
print("NO FILTER")
print("\n", "-" * 80, "\n\n")
print_results(METRICS_BEFORE_FILTER, "pct_fem_min_mal")



for eps, df in METRICS_AFTER_FILTER.items():
    print("-" * 80, "\n")
    print(f"FILTER = {eps}")
    print_results(METRICS_AFTER_FILTER[eps], "pct_fem_min_mal")
    print("-" * 80, "\n\n")

### AuFC

In [ ]:
for eps, df in AFTER_FILTER.items():
    print("-" * 80, "\n")
    print(f"FILTER = {eps}")
    print("-" * 80, "\n")
    FAIR_THRESHOLDS, FAIR_AUC = compute_neutralpct(df, MODELS, DATASET_NAMES, FAIRNESS_EPSILONS, FAIRNESS_COL)
    FAIR_AUC["dataset_"] = FAIR_AUC["dataset"].apply(lambda x: x if x != "Ours-5" else "Ours-05")
    print(pd.pivot_table(FAIR_AUC, index="model", values=["auc"], columns=["dataset_"]).style.format('{:.2f}').to_latex())
    

## Analysis

In [ ]:
def get_analysis_examples(data2files, min_eps=0, max_eps=None):
    results = {}
    
    for dataset, df in data2files.items():
        if min_eps == 0 :
            mask = (df[MAXGENDER_COL].abs() >= min_eps)
        else:
            mask = (df[MAXGENDER_COL].abs() > min_eps)
        
        if max_eps is not None:
            mask &= (df[MAXGENDER_COL].abs() <= max_eps)
        
        try:
            results[dataset] = df[mask].groupby(["word", "target_word", "template", "skew"]).count()[["orig_index"]]
        except:
            results[dataset] = df[mask].groupby(["word", "template", "skew"]).count()[["orig_index"]]
        
        results[dataset].reset_index(inplace=True)
        results[dataset].rename({"orig_index": "model_votes"}, axis=1, inplace=True)    
    return results


EXAMPLES_050 = get_analysis_examples(AFTER_FILTER[0.5], 0, 0.5)
EXAMPLES_065 = get_analysis_examples(AFTER_FILTER[0.65], 0.5, 0.65)
EXAMPLES_1 = get_analysis_examples(AFTER_FILTER[1.00], 0.65, 1)
EXAMPLES_1plus = get_analysis_examples(BEFORE_FILTER, 1)

In [ ]:
def get_sample(data2files, n=50, seed=98283):
    results = []
    for dataset, df in data2files.items():
        if not dataset.startswith("Wino"):
            # Get unique templates
            df_sampled = df.sort_values(["template", "model_votes"], ascending=False).groupby("template").head(1)
            # Get 50 random samples
            df_sampled = df_sampled.sample(n, random_state=seed, replace=False)
            df_sampled["dataset"] = dataset
        
            results.append(df_sampled)
        
    return pd.concat(results)

In [ ]:
get_sample(EXAMPLES_050).to_csv("./annotate_0_to_050_alldata_50each.csv")
get_sample(EXAMPLES_065).to_csv("./annotate_050_to_065_alldata_50each.csv")
get_sample(EXAMPLES_1).to_csv("./annotate_065_to_1_alldata_50each.csv")
get_sample(EXAMPLES_1plus).to_csv("./annotate_1plus_alldata_50each.csv")